# Combine All Datasets: Demographics + Weather + Livestock

## Purpose
This notebook combines three datasets to create the final dataset for life expectancy modeling:

1. **Demographics Dataset**: Cleaned life expectancy + ACS demographic/economic variables (from notebook 04)
2. **Weather Dataset**: Environmental and air quality variables (2012-2019)
3. **Livestock Dataset**: Agricultural livestock counts by county (2012-2019)

All three datasets are merged on **FIPS codes** (5-digit county identifier) and **Year** using **inner join** to keep only counties with complete data across all sources.

## Data Loss Tracking
At each merge step, we track:
- Starting number of observations
- Number of observations after merge
- Number of observations lost
- Percentage of data retained

## Input Files
- `data_cleaned/demographics_final/combined_all_years_cleaned_final.csv` (Life expectancy + ACS variables)
- `data_cleaned/weather/final_dataset_103_features.pkl` (Weather/environmental data with engineered features)
- `data_cleaned/livestock/county_mean_YYYY.csv` (8 files, 2012-2019)

## Output
- `data_cleaned/combined_final/final_combined_all_variables.csv`
- Final dataset ready for ML modeling with ~100+ features

## 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

## 2. Load Demographics Dataset (Cleaned from Notebook 04)

This dataset already has:
- Life expectancy (target variable)
- 19 ACS demographic/economic/health variables
- 6 calculated percentage features (race, housing, family)
- Years: 2012-2019
- FIPS codes: State_FIPS + County_FIPS

In [2]:
# Load cleaned demographics dataset from notebook 04
demographics_df = pd.read_csv('../data_cleaned/demographics_final/combined_all_years_cleaned_final.csv', 
                              dtype={'State_FIPS': str, 'County_FIPS': str})

print("=" * 70)
print("DEMOGRAPHICS DATASET LOADED")
print("=" * 70)
print(f"Shape: {demographics_df.shape}")
print(f"  - Rows (county-year observations): {demographics_df.shape[0]:,}")
print(f"  - Columns: {demographics_df.shape[1]}")
print(f"  - Years: {sorted(demographics_df['Year'].unique())}")
print(f"  - Unique counties: {demographics_df.groupby('Year').size().mean():.0f} per year (average)")

print("\nColumns:")
print(demographics_df.columns.tolist())

DEMOGRAPHICS DATASET LOADED
Shape: (24871, 21)
  - Rows (county-year observations): 24,871
  - Columns: 21
  - Years: [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
  - Unique counties: 3109 per year (average)

Columns:
['County', 'State', 'Mean Life Expectancy', 'State_FIPS', 'County_FIPS', 'Poverty Rate', 'High School Degree or Higher (%)', "Bachelor's Degree or Higher (%)", 'Median Age', 'Gini Index', 'Disability Rate', 'Total Population', 'Median Household Income', 'Unemployment Rate', 'Year', 'White Population (%)', 'Hispanic Population (%)', 'Black Population (%)', 'Households with No Vehicle (%)', 'Rent Burden (+50% of HI)', 'Single Mother Families (%)']


In [3]:
# Display first few rows
print("\nSample data:")
demographics_df.head()


Sample data:


,County,State,Mean Life Expectancy,State_FIPS,County_FIPS,Poverty Rate,High School Degree or Higher (%),Bachelor's Degree or Higher (%),Median Age,Gini Index,...,Total Population,Median Household Income,Unemployment Rate,Year,White Population (%),Hispanic Population (%),Black Population (%),Households with No Vehicle (%),Rent Burden (+50% of HI),Single Mother Families (%)
0,Autauga County,Alabama,75.729244,1,1,11.6,85.0,24.4,37.0,0.4122,...,54590.0,53773.0,8.6,2012,78.922880,2.399707,18.098553,5.136952,20.377868,10.955253
1,Baldwin County,Alabama,77.927178,1,3,13.3,87.0,29.3,41.2,0.4364,...,183226.0,50706.0,8.5,2012,86.373659,4.319802,9.286892,3.081745,18.386173,8.876182
2,Barbour County,Alabama,75.726346,1,5,26.1,70.2,13.0,38.2,0.4758,...,27469.0,31889.0,13.5,2012,48.957006,4.969238,46.033711,9.975592,25.047801,16.343447
3,Bibb County,Alabama,73.854904,1,7,16.5,71.5,8.2,39.4,0.4212,...,22769.0,36824.0,10.5,2012,76.678818,1.840221,21.753261,5.117790,16.062544,9.766764
4,Blount County,Alabama,75.703162,1,9,14.7,73.9,12.0,39.1,0.4112,...,57466.0,45192.0,10.0,2012,94.850868,8.084781,1.312080,3.832438,16.381480,6.856960


## 3. Create FIPS Code for Demographics Dataset

Combine State_FIPS and County_FIPS into a single 5-digit FIPS code for merging.
- State_FIPS: 2 digits (e.g., "01" for Alabama)
- County_FIPS: 3 digits (e.g., "001" for Autauga County)
- Combined FIPS: "01001"

In [4]:
# Ensure proper zero-padding for FIPS codes
demographics_df['State_FIPS'] = demographics_df['State_FIPS'].str.zfill(2)  # Ensure 2-digit state FIPS
demographics_df['County_FIPS'] = demographics_df['County_FIPS'].str.zfill(3)  # Ensure 3-digit county FIPS

# Combine into single FIPS column
demographics_df['fips'] = demographics_df['State_FIPS'] + demographics_df['County_FIPS']

print("FIPS codes created for demographics dataset:")
print(f"  - Example FIPS codes: {demographics_df['fips'].head(3).tolist()}")
print(f"  - Total unique FIPS codes: {demographics_df['fips'].nunique()}")

FIPS codes created for demographics dataset:
  - Example FIPS codes: ['01001', '01003', '01005']
  - Total unique FIPS codes: 3111


## 4. Load Weather Dataset

The weather dataset (`final_dataset_103_features.pkl`) contains environmental and air quality variables:
- Total AOD (Aerosol Optical Depth) at multiple wavelengths
- Total column measurements (CO, CH4, NO2, O3, SO2, etc.)
- Vertically integrated mass of aerosols (dust, black carbon, sea salt, etc.)
- FoT (Frequency of Time) threshold variables for air quality and temperature
- Years: 2010-2019 (we'll filter to 2012-2019 to match demographics data)

In [5]:
# Load weather data from pickle file with engineered features
weather_df = pd.read_pickle('../data_cleaned/weather/final_dataset_103_features.pkl')

print("=" * 70)
print("WEATHER DATASET LOADED")
print("=" * 70)
print(f"Shape: {weather_df.shape}")
print(f"  - Rows: {weather_df.shape[0]:,}")
print(f"  - Columns: {weather_df.shape[1]}")
print(f"  - Years available: {sorted(weather_df['year'].unique())}")

# Filter to 2012-2019 to match demographics data
weather_df = weather_df[weather_df['year'] >= 2012]
print(f"\nAfter filtering to 2012-2019:")
print(f"  - Rows: {weather_df.shape[0]:,}")
print(f"  - Unique FIPS codes: {weather_df['fips'].nunique()}")

WEATHER DATASET LOADED
Shape: (52320, 106)
  - Rows: 52,320
  - Columns: 106
  - Years available: [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

After filtering to 2012-2019:
  - Rows: 24,618
  - Unique FIPS codes: 3079


In [6]:
# Display weather dataset info
print("\nWeather dataset columns (first 20):")
print(weather_df.columns.tolist()[:20])

print("\nSample weather data:")
weather_df.head()


Weather dataset columns (first 20):
['fips', 'year', 'MeanLifeExpectency', '2m dew point temperature', '2m temperature', 'Black carbon AOD at 550 nm', 'Dust AOD at 550 nm', 'Land-sea mask', 'Mean sea level pressure', 'Organic matter AOD at 550 nm', 'PM$_1$', 'PM$_{2.5}$', 'PM$_{10}$', 'Sea salt AOD at 550 nm', 'Sulphate AOD at 550 nm', 'Surface geopotential', 'Surface pressure', 'Total AOD at 469 nm', 'Total AOD at 550 nm', 'Total AOD at 670 nm']

Sample weather data:


,fips,year,MeanLifeExpectency,2m dew point temperature,2m temperature,Black carbon AOD at 550 nm,Dust AOD at 550 nm,Land-sea mask,Mean sea level pressure,Organic matter AOD at 550 nm,...,FoT PM$_{10}$ above EPA threshold,FoT PM$_{1}$ above75ᵗʰ percentile,FoT PM$_{2.5}$ above75ᵗʰ percentile,FoT Propane above75ᵗʰ percentile,FoT Sulphur dioxide above75ᵗʰ percentile,FoT Temperature above75ᵗʰ percentile,FoT Temperature above 90 $\mathrm{^o F}$,FoT Temperature below25ᵗʰ percentile,FoT Temperature below 0 $\mathrm{^o C}$,FoT PM$_{2.5}$ above EPA threshold
27702,1001,2012,75.729244,285.513501,292.554172,0.00723,0.011407,0.986498,101758.939674,0.078036,...,0.000000,42.588798,43.647541,24.795082,56.523224,59.016393,2.493169,17.042350,0.751366,62.841530
27703,1003,2012,77.927178,288.958325,293.661494,0.007093,0.01424,0.625803,101740.052225,0.072943,...,0.102459,34.699454,39.105191,14.310109,30.327869,65.915301,0.990437,10.075137,0.102459,56.010929
27704,1005,2012,75.726346,285.695067,292.914944,0.007026,0.011077,0.987906,101764.617155,0.073908,...,0.068306,36.612022,39.002732,20.252732,11.475410,62.021858,2.219945,14.139344,0.546448,61.577869
27705,1007,2012,73.854904,285.539025,292.150932,0.007252,0.011192,0.987537,101757.718522,0.078939,...,0.000000,43.203552,44.023224,27.117486,59.187158,56.796448,2.322404,18.613388,0.956284,62.739071
27706,1009,2012,75.703162,284.972011,291.345408,0.007078,0.009757,0.9805,101770.881936,0.078042,...,0.000000,43.715847,43.852459,32.411202,62.773224,52.937158,1.673497,21.755464,1.161202,65.198087


## 5. Standardize FIPS Codes in Weather Dataset

Ensure FIPS codes are properly formatted as 5-digit strings with zero-padding.

In [7]:
# Convert FIPS to string and ensure 5-digit format
weather_df['fips'] = weather_df['fips'].astype(str).str.zfill(5)

# Create Year column to match demographics dataset
weather_df['Year'] = weather_df['year']

print("FIPS codes standardized for weather dataset:")
print(f"  - Example FIPS codes: {weather_df['fips'].head(3).tolist()}")
print(f"  - Unique FIPS codes: {weather_df['fips'].nunique()}")
print(f"  - Years: {sorted(weather_df['Year'].unique())}")

FIPS codes standardized for weather dataset:
  - Example FIPS codes: ['01001', '01003', '01005']
  - Unique FIPS codes: 3079
  - Years: [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]


## 6. Load Livestock Dataset

The livestock dataset contains county-level livestock counts by animal type:
- Buffalo, Cattle, Chicken, Duck, Goat, Horse, Pig, Sheep
- Files: county_mean_2012.csv through county_mean_2019.csv
- Need to concatenate all years into single dataset

In [8]:
def combine_livestock_files(directory, years):
    """
    Reads multiple livestock CSV files and concatenates them into a single DataFrame.
    
    Parameters:
        directory (str): Path to folder containing livestock CSV files
        years (list): List of years to load (e.g., [2012, 2013, ..., 2019])
    
    Returns:
        pd.DataFrame: Combined dataframe with all years
    """
    all_data = []
    
    for year in years:
        file_path = os.path.join(directory, f"county_mean_{year}.csv")
        
        # Load CSV and convert FIPS columns to strings
        df = pd.read_csv(file_path)
        df['STATEFP'] = df['STATEFP'].astype(str)
        df['COUNTYFP'] = df['COUNTYFP'].astype(str)
        
        all_data.append(df)
        print(f"  Loaded {year}: {df.shape[0]:,} rows")
    
    # Combine all years
    combined_df = pd.concat(all_data, ignore_index=True)
    return combined_df

# Load and combine livestock data
print("=" * 70)
print("LOADING LIVESTOCK DATASET")
print("=" * 70)

directory = "../data_cleaned/livestock/"
years = [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

livestock_df = combine_livestock_files(directory, years)

print(f"\nCombined livestock dataset:")
print(f"  - Total rows: {livestock_df.shape[0]:,}")
print(f"  - Columns: {livestock_df.shape[1]}")

LOADING LIVESTOCK DATASET
  Loaded 2012: 3,197 rows
  Loaded 2013: 3,197 rows
  Loaded 2014: 3,197 rows
  Loaded 2015: 3,197 rows
  Loaded 2016: 3,197 rows
  Loaded 2017: 3,197 rows
  Loaded 2018: 3,197 rows
  Loaded 2019: 3,197 rows

Combined livestock dataset:
  - Total rows: 25,576
  - Columns: 13


In [9]:
# Display livestock dataset info
print("Livestock variables:")
print(livestock_df.columns.tolist())

print("\nSample livestock data:")
livestock_df.head()

Livestock variables:
['STATEFP', 'COUNTYFP', 'NAME', 'GEOID', 'year', 'Buffalo', 'Cattle', 'Chicken', 'Duck', 'Goat', 'Horse', 'Pig', 'Sheep']

Sample livestock data:


,STATEFP,COUNTYFP,NAME,GEOID,year,Buffalo,Cattle,Chicken,Duck,Goat,Horse,Pig,Sheep
0,1,1,Autauga,1001,2012,104.304193,765.477347,1460.733157,6.200551,47.556840,54.780249,3.199114,13.548855
1,1,3,Baldwin,1003,2012,0.000000,356.486268,20.764893,2.339871,20.358615,74.033112,3.038270,4.505194
2,1,5,Barbour,1005,2012,74.777062,595.600612,150608.306932,6.572719,11.807808,24.807758,8.248660,6.213882
3,1,7,Bibb,1007,2012,0.000000,288.782290,516.739571,3.657299,15.416166,42.510005,20.036976,2.450206
4,1,9,Blount,1009,2012,0.000000,1692.635794,549287.154544,16.943384,42.096943,228.072444,9.920152,22.714804


## 7. Create FIPS Codes for Livestock Dataset

Standardize FIPS codes to match demographics and weather datasets.

In [10]:
# Ensure proper zero-padding for FIPS codes
livestock_df['STATEFP'] = livestock_df['STATEFP'].str.zfill(2)  # 2-digit state
livestock_df['COUNTYFP'] = livestock_df['COUNTYFP'].str.zfill(3)  # 3-digit county

# Combine into single FIPS column
livestock_df['fips'] = livestock_df['STATEFP'] + livestock_df['COUNTYFP']

# Create Year column (rename from 'year')
livestock_df['Year'] = livestock_df['year']

print("FIPS codes created for livestock dataset:")
print(f"  - Example FIPS codes: {livestock_df['fips'].head(3).tolist()}")
print(f"  - Unique FIPS codes: {livestock_df['fips'].nunique()}")
print(f"  - Years: {sorted(livestock_df['Year'].unique())}")

FIPS codes created for livestock dataset:
  - Example FIPS codes: ['01001', '01003', '01005']
  - Unique FIPS codes: 3197
  - Years: [2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]


## 8. Merge Demographics + Weather Datasets

First merge: Combine demographics dataset with weather dataset on FIPS and Year.
Using **inner join** to keep only counties that exist in both datasets.

In [11]:
# Track starting observations
print("=" * 70)
print("MERGE STEP 1: DEMOGRAPHICS + WEATHER")
print("=" * 70)

demographics_initial_rows = len(demographics_df)
weather_rows = len(weather_df)

print(f"Demographics dataset rows: {demographics_initial_rows:,}")
print(f"Weather dataset rows: {weather_rows:,}")

# Perform inner join on fips and Year
merged_step1 = pd.merge(demographics_df, weather_df, on=['fips', 'Year'], how='inner')

# Calculate data loss
merged_step1_rows = len(merged_step1)
rows_lost = demographics_initial_rows - merged_step1_rows
retention_pct = (merged_step1_rows / demographics_initial_rows) * 100

print(f"\nAfter merge:")
print(f"  - Rows: {merged_step1_rows:,}")
print(f"  - Rows lost: {rows_lost:,} ({100 - retention_pct:.2f}%)")
print(f"  - Retention: {retention_pct:.2f}%")
print(f"  - Columns: {merged_step1.shape[1]}")

MERGE STEP 1: DEMOGRAPHICS + WEATHER
Demographics dataset rows: 24,871
Weather dataset rows: 24,618

After merge:
  - Rows: 24,604
  - Rows lost: 267 (1.07%)
  - Retention: 98.93%
  - Columns: 127


In [12]:
# Display merged dataset info
print("\nMerged dataset (Demographics + Weather) sample:")
merged_step1.head()


Merged dataset (Demographics + Weather) sample:


,County,State,Mean Life Expectancy,State_FIPS,County_FIPS,Poverty Rate,High School Degree or Higher (%),Bachelor's Degree or Higher (%),Median Age,Gini Index,...,FoT PM$_{10}$ above EPA threshold,FoT PM$_{1}$ above75ᵗʰ percentile,FoT PM$_{2.5}$ above75ᵗʰ percentile,FoT Propane above75ᵗʰ percentile,FoT Sulphur dioxide above75ᵗʰ percentile,FoT Temperature above75ᵗʰ percentile,FoT Temperature above 90 $\mathrm{^o F}$,FoT Temperature below25ᵗʰ percentile,FoT Temperature below 0 $\mathrm{^o C}$,FoT PM$_{2.5}$ above EPA threshold
0,Autauga County,Alabama,75.729244,01,001,11.6,85.0,24.4,37.0,0.4122,...,0.000000,42.588798,43.647541,24.795082,56.523224,59.016393,2.493169,17.042350,0.751366,62.841530
1,Baldwin County,Alabama,77.927178,01,003,13.3,87.0,29.3,41.2,0.4364,...,0.102459,34.699454,39.105191,14.310109,30.327869,65.915301,0.990437,10.075137,0.102459,56.010929
2,Barbour County,Alabama,75.726346,01,005,26.1,70.2,13.0,38.2,0.4758,...,0.068306,36.612022,39.002732,20.252732,11.475410,62.021858,2.219945,14.139344,0.546448,61.577869
3,Bibb County,Alabama,73.854904,01,007,16.5,71.5,8.2,39.4,0.4212,...,0.000000,43.203552,44.023224,27.117486,59.187158,56.796448,2.322404,18.613388,0.956284,62.739071
4,Blount County,Alabama,75.703162,01,009,14.7,73.9,12.0,39.1,0.4112,...,0.000000,43.715847,43.852459,32.411202,62.773224,52.937158,1.673497,21.755464,1.161202,65.198087


## 9. Merge with Livestock Dataset

Second merge: Add livestock data to the combined demographics+weather dataset.
Using **inner join** to keep only counties with complete data across all three sources.

In [13]:
# Track starting observations
print("=" * 70)
print("MERGE STEP 2: (DEMOGRAPHICS + WEATHER) + LIVESTOCK")
print("=" * 70)

step1_rows = len(merged_step1)
livestock_rows = len(livestock_df)

print(f"Demographics+Weather rows: {step1_rows:,}")
print(f"Livestock dataset rows: {livestock_rows:,}")

# Perform inner join on fips and Year
final_merged = pd.merge(merged_step1, livestock_df, on=['fips', 'Year'], how='inner')

# Calculate data loss
final_rows = len(final_merged)
rows_lost_step2 = step1_rows - final_rows
total_rows_lost = demographics_initial_rows - final_rows
final_retention_pct = (final_rows / demographics_initial_rows) * 100

print(f"\nAfter merge:")
print(f"  - Rows: {final_rows:,}")
print(f"  - Rows lost in this step: {rows_lost_step2:,}")
print(f"  - Total rows lost from original: {total_rows_lost:,}")
print(f"  - Final retention: {final_retention_pct:.2f}%")
print(f"  - Columns: {final_merged.shape[1]}")

MERGE STEP 2: (DEMOGRAPHICS + WEATHER) + LIVESTOCK
Demographics+Weather rows: 24,604
Livestock dataset rows: 25,576

After merge:
  - Rows: 24,487
  - Rows lost in this step: 117
  - Total rows lost from original: 384
  - Final retention: 98.46%
  - Columns: 140


In [14]:
# Check for duplicate columns
print("\nChecking for duplicate columns:")
duplicate_cols = [col for col in final_merged.columns if final_merged.columns.tolist().count(col) > 1]
if duplicate_cols:
    print(f"  - Duplicate columns found: {duplicate_cols}")
else:
    print("  - No duplicate columns")

# Display column names
print(f"\nAll columns ({len(final_merged.columns)}):")
for i, col in enumerate(final_merged.columns, 1):
    print(f"  {i:3}. {col}")


Checking for duplicate columns:
  - Duplicate columns found: ['Specific humidity', 'Specific humidity']

All columns (140):
    1. County
    2. State
    3. Mean Life Expectancy
    4. State_FIPS
    5. County_FIPS
    6. Poverty Rate
    7. High School Degree or Higher (%)
    8. Bachelor's Degree or Higher (%)
    9. Median Age
   10. Gini Index
   11. Disability Rate
   12. Total Population
   13. Median Household Income
   14. Unemployment Rate
   15. Year
   16. White Population (%)
   17. Hispanic Population (%)
   18. Black Population (%)
   19. Households with No Vehicle (%)
   20. Rent Burden (+50% of HI)
   21. Single Mother Families (%)
   22. fips
   23. year_x
   24. MeanLifeExpectency
   25. 2m dew point temperature
   26. 2m temperature
   27. Black carbon AOD at 550 nm
   28. Dust AOD at 550 nm
   29. Land-sea mask
   30. Mean sea level pressure
   31. Organic matter AOD at 550 nm
   32. PM$_1$
   33. PM$_{2.5}$
   34. PM$_{10}$
   35. Sea salt AOD at 550 nm
   36. Su

## 10. Clean Up Redundant Columns

Remove duplicate identifier columns that were created during merges:
- Keep: County, State, Year, fips, Mean Life Expectancy
- Drop: State_FIPS, County_FIPS, STATEFP, COUNTYFP, GEOID, NAME, year (lowercase)

In [ ]:
print("=" * 70)
print("DROPPING REDUNDANT COLUMNS")
print("=" * 70)

# List of columns to drop
columns_to_drop = []

# Check which columns exist before dropping
potential_drops = ['State_FIPS', 'County_FIPS', 'STATEFP', 'COUNTYFP', 
                   'GEOID', 'NAME', 'year', 'year_y', 'year_x', 'fips', 'MeanLifeExpectency']

for col in potential_drops:
    if col in final_merged.columns:
        columns_to_drop.append(col)
        print(f"  Will drop: {col}")

# Drop the columns
if columns_to_drop:
    final_merged = final_merged.drop(columns=columns_to_drop)
    print(f"\n✓ Dropped {len(columns_to_drop)} redundant columns")
else:
    print("  No redundant columns to drop")

print(f"\nFinal dataset shape: {final_merged.shape}")
print(f"  - Rows: {final_merged.shape[0]:,}")
print(f"  - Columns: {final_merged.shape[1]}")

## 11. Final Data Quality Checks

Verify the final dataset integrity before saving.

In [ ]:
print("=" * 70)
print("FINAL DATA QUALITY CHECKS")
print("=" * 70)

# Check for missing values
missing_values = final_merged.isnull().sum()
total_missing = missing_values.sum()

print(f"Missing values: {total_missing}")
if total_missing > 0:
    print("\nColumns with missing values:")
    for col in final_merged.columns:
        if missing_values[col] > 0:
            pct = (missing_values[col] / len(final_merged)) * 100
            print(f"  {col}: {missing_values[col]:,} ({pct:.2f}%)")
else:
    print("   No missing values")

# Check year distribution
print(f"\nObservations per year:")
year_counts = final_merged['Year'].value_counts().sort_index()
for year, count in year_counts.items():
    print(f"  {year}: {count:,} counties")

# Check for duplicate rows
duplicates = final_merged.duplicated(subset=['fips', 'Year']).sum()
print(f"\nDuplicate county-year pairs: {duplicates}")
if duplicates > 0:
    print("   Warning: Duplicate rows detected!")
else:
    print("   No duplicates")

# Display key statistics
print(f"\nKey variable statistics:")
print(f"  Target variable: Mean Life Expectancy")
print(f"    Min: {final_merged['Mean Life Expectancy'].min():.2f}")
print(f"    Max: {final_merged['Mean Life Expectancy'].max():.2f}")
print(f"    Mean: {final_merged['Mean Life Expectancy'].mean():.2f}")
print(f"    Median: {final_merged['Mean Life Expectancy'].median():.2f}")

In [ ]:
# Display sample of final dataset
print("\nFinal dataset sample:")
final_merged.head(10)

In [ ]:
# Display final column list
print(f"\nFinal dataset columns ({len(final_merged.columns)}):")
print("=" * 70)

# Group columns by category for clarity
identifiers = ['County', 'State', 'fips', 'Year']
target = ['Mean Life Expectancy']

# ACS variables (original + engineered)
acs_vars = [col for col in final_merged.columns if col in [
    'Median Household Income', 'Total Population', 'Gini Index', 'Median Age',
    'Poverty Rate', 'Unemployment Rate', 'Disability Rate',
    "Bachelor's Degree or Higher (%)", "High School Degree or Higher (%)",
    'White Population (%)', 'Hispanic Population (%)', 'Black Population (%)',
    'Households with No Vehicle (%)', 'Rent Burden (+50% of HI)', 'Single Mother Families (%)'
]]

livestock_vars = [col for col in final_merged.columns if col in [
    'Buffalo', 'Cattle', 'Chicken', 'Duck', 'Goat', 'Horse', 'Pig', 'Sheep'
]]

weather_vars = [col for col in final_merged.columns if col not in identifiers + target + acs_vars + livestock_vars]

print("\nIdentifiers:")
for col in identifiers:
    if col in final_merged.columns:
        print(f"  - {col}")

print("\nTarget Variable:")
for col in target:
    if col in final_merged.columns:
        print(f"  - {col}")

print(f"\nACS Variables ({len(acs_vars)}):")
for col in acs_vars:
    print(f"  - {col}")

print(f"\nLivestock Variables ({len(livestock_vars)}):")
for col in livestock_vars:
    print(f"  - {col}")

print(f"\nWeather/Environmental Variables ({len(weather_vars)}):")
for col in weather_vars[:10]:  # Show first 10 only
    print(f"  - {col}")
if len(weather_vars) > 10:
    print(f"  ... and {len(weather_vars) - 10} more weather variables")

## 12. Save Final Combined Dataset

Export the final dataset to CSV for use in ML modeling notebooks.

In [ ]:
# Create output directory if needed
output_dir = Path('../data_cleaned/combined_final')
output_dir.mkdir(parents=True, exist_ok=True)

# Save final combined dataset
output_path = output_dir / 'final_combined_all_variables.csv'
final_merged.to_csv(output_path, index=False)

print("=" * 70)
print("FINAL DATASET SAVED")
print("=" * 70)
print(f" File: {output_path}")
print(f"  - Shape: {final_merged.shape}")
print(f"  - Rows: {final_merged.shape[0]:,}")
print(f"  - Columns: {final_merged.shape[1]}")
print(f"  - File size: {output_path.stat().st_size / (1024*1024):.2f} MB")
print(f"  - Years: {sorted(final_merged['Year'].unique())}")
print(f"  - Counties per year: {final_merged.groupby('Year').size().mean():.0f} (average)")

## 13. Summary

**Data Merging Process:**
1. Loaded demographics dataset (notebook 04 output)
2. Loaded weather dataset (final_dataset_103_features.pkl, filtered to 2012-2019)
3. Loaded livestock dataset (8 yearly files combined)
4. Standardized FIPS codes across all datasets
5. Merged demographics + weather (inner join on fips + Year)
6. Merged with livestock (inner join on fips + Year)
7. Removed redundant columns
8. Verified data quality

**Final Dataset:**
- **Observations**: Check output above (county-year pairs)
- **Years**: 2012-2019 (8 years)
- **Target variable**: Mean Life Expectancy
- **Feature categories**:
  - ACS demographic/economic/education variables (~15)
  - Engineered percentage features (6)
  - Livestock counts (8 animal types)
  - Weather/environmental variables (~100+)
- **Total features**: ~120+ variables

**Data Retention:**
- Check output above for exact retention percentage
- Used inner join to ensure complete data across all sources
- Only counties with all three data types are included

**Next Steps:**
- Proceed to notebook 06 for ML modeling (Random Forest, XGBoost, etc.)
- Consider feature selection to identify most important predictors
- Ready for machine learning model development